# Claude Collection

In [ ]:
#### API Menu

#### 1 claude.messages.create
#### item:
    model='claude-3-haiku-20240307'
           #'claude-3-5-sonnet-20240620'
    system='system setting'

    messages=[
        {'role': 'user', 'content': x},
    ]
    
    max_tokens=200

    temperature=.7
#### return:
    res.content[0].text
#### 2 claude.messages.stream
#### item:
    model='claude-3-haiku-20240307'

    system=''''
    You are an assistant that reimplements Python code in high performance C++ for an M3 Mac. 
    Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. 
    The C++ response needs to produce an identical output in the fastest possible time.
    '''
    
    messages=[{
        'role': 'user', 
        'content': '''
            Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. 
            Respond only with C++ code; do not explain your work other than a few comments. 
            Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.
            + code
        '''
    }]

    max_tokens=200

#### return:
    res = ''
    with llm as stream:
        for y in stream.text_stream:
            res += y

#### 2 hf.text_generation
#### hf setting:
    provider='hf-inference'
	api_key='hf+token'
#### item:
	model='google/gemma-2-2b-it'
	inputs='Can you please let us know more details about your '
	provider='hf-inference'
#### return:

# 1 Answer with set system and model

In [5]:
import os
import anthropic
from dotenv import load_dotenv
load_dotenv()
claude = anthropic.Anthropic()
def say(x):
    res = claude.messages.create(
        model='claude-3-haiku-20240307', # it's cheapest. high quality: claude-3-5-sonnet-latest
        max_tokens=200,
        temperature=0.7,
        system='Please answer one short and simple sentence.',
        messages=[
            {'role': 'user', 'content': x},
        ],
    )
    return res.content[0].text
print(say('Hello. How are you doing?'))

I'm doing well, thank you.


# 2 Conversations

In [13]:
import os
from dotenv import load_dotenv
import anthropic
load_dotenv()
claude = anthropic.Anthropic()
msgsA = ['Hi there']
msgsB = ['Hi']
def sayA():
    msg = []
    for x, y in zip(msgsA, msgsB):
        msg.append({'role': 'assistant', 'content': x})
        msg.append({'role': 'user', 'content': y})
    res = claude.messages.create(
        model= 'claude-3-haiku-20240307',
        system='You are not an assistant. Please treat other one like your friend. Also, please answer with short sentence.',
        messages=msg,
        max_tokens=500
    )
    return res.content[0].text
def sayB():
    msg = []
    for x, y in zip(msgsA, msgsB):
        msg.append({'role': 'user', 'content': x})
        msg.append({'role': 'assistant', 'content': y})
    msg.append({'role': 'user', 'content': msgsA[-1]})
    res = claude.messages.create(
        model= 'claude-3-haiku-20240307',
        system='You are not an assistant. Please treat other one like your friend. Also, please answer with short sentence.',
        messages=msg,
        max_tokens=500
    )
    return res.content[0].text

for i in range(5):
    a_next = sayA()
    print(f"A:{a_next}\n")
    msgsA.append(a_next)
    b_next = sayB()
    print(f"B:{b_next}\n")
    msgsB.append(b_next)
    

A:It's nice to meet you! How has your day been going so far?

B:My day has been going well, thanks for asking! How about you, how has your day been?

A:My day has been good, thanks for asking! I've enjoyed our conversation so far.

B:That's great to hear! I've enjoyed our conversation as well. Is there anything in particular you'd like to chat about?

A:I'm always happy to discuss a variety of topics! What kind of things are you interested in? I enjoy intellectual discussions on subjects like science, philosophy, arts and culture.

B:I'm always happy to discuss a wide range of topics! Some of my favorite areas are science, philosophy, and the arts. I find intellectual discussions on these subjects to be really fascinating. What in particular interests you the most?

A:I find science and philosophy to be especially interesting topics. I'm always eager to learn more about the latest discoveries and theories in fields like physics, biology, and cognitive science. And I'm fascinated by big

# 3 Generate Code

In [21]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o generated_cpp generated_cpp.cpp
!./generated_cpp

generated_cpp.cpp:15:57: error: no member named 'setprecision' in namespace 'std'
    std::cout << "Running time: " << std::fixed << std::setprecision(6) << std::chrono::duration<double>(end_time - start_time).count() << "s" << std::endl;
                                                   ~~~~~^
1 error generated.
hi
Result: 
8.542e-06


In [124]:
import os
import io
import sys
import anthropic
import time
import subprocess
from dotenv import load_dotenv
load_dotenv()
claude = anthropic.Anthropic()
def sys_settting():
    x = 'You are an assistant that reimplements Python code in high performance C++ for an M1 Mac. '
    x += 'Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. '
    x += 'The C++ response needs to produce an identical output in the fastest possible time.'
    return x
def usr_settting_and_code(x1):
    x2 = 'Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. '
    x2 += 'Respond only with C++ code; do not explain your work other than a few comments. '
    x2 += 'Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n'
    x2 += x1
    return x2
def write_output(x):
    code = x.replace('```cpp','').replace('```','')
    with open('code.cpp', 'w') as f:
        f.write(code)
def py_to_cpp_claude(x):
    result = claude.messages.stream(
        model='claude-3-haiku-20240307',
        max_tokens=2000,
        system=sys_settting(),
        messages=[{'role': 'user', 'content': usr_settting_and_code(x)}],
    )
    reply = ''
    with result as stream:
        for text in stream.text_stream:
            reply += text
            print(text, end="", flush=True)
    write_output(reply)
# def py_to_cpp_claude(x):
#     llm = claude.messages.stream(
#         model='claude-3-haiku-20240307',
#         max_tokens=200,
#         system=sys_settting(),
#         messages=[{'role': 'user', 'content': usr_settting_and_code(x)}],
#     )
#     x3 = ''
#     with llm as stream:
#         for y in stream.text_stream:
#             x3 += y
#             yield reply.replace('```cpp\n','').replace('```','')
#     write_output(x3)
#     return x3
# def py_to_cpp_claude(x):
#     result = claude.messages.stream(
#         model='claude-3-haiku-20240307',
#         max_tokens=2000,
#         system=system_message,
#         messages=[{'role': 'user', 'content': usr_settting_and_code(x)}],
#     )
#     reply = ''
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             print(text, end='', flush=True)
#     write_output(reply)
# def py_to_cpp_claude(x):
#     result = claude.messages.stream(
#         model='claude-3-haiku-20240307',
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": usr_settting_and_code(x)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             yield reply.replace('```cpp\n','').replace('```','')
def run_cpp():
        try:
            run1 = subprocess.run(['clang++', '-Ofast', '-std=c++17', '-march=armv8.5-a', '-mtune=apple-m1', '-mcpu=apple-m1', '-o', 'code', 'code.cpp'], check=True, text=True, capture_output=True)
            run2 = subprocess.run(['./code'], check=True, text=True, capture_output=True)
            return run2.stdout
        except subprocess.CalledProcessError as e:
            return f'An error occurred:\n{e.stderr}'

In [132]:
code2 = """
import time
def speak():
    return 'kiwi'
start_time = time.time()
res = speak()
print('Result:', res)
"""
code3 ="""
from datetime import datetime
def count(x,y):
    for n in range(x,y):
        print(n)
start_time = datetime.now()
count(1,11)
end_time = datetime.now()
"""
code4 = """
import time
def say():
    return 'mongo'
start_time_in_sec = time.time()
res= say()
end_time_in_sec = time.time()
# print(f'Result: {result:.12f}')
print(f'Execution Time: {(end_time_in_sec-start_time_in_sec):.6f} seconds')
print(res, end_time_in_sec-start_time_in_sec)
"""
print('------------------')
print(code4)
print('------------------')
exec(code4)
# cpp show and run code(claude)
print('------------------')
print(py_to_cpp_claude(code4))
print('------------------')
print(run_cpp())

------------------

import time
def say():
    return 'mongo'
start_time_in_sec = time.time()
res= say()
end_time_in_sec = time.time()
# print(f'Result: {result:.12f}')
print(f'Execution Time: {(end_time_in_sec-start_time_in_sec):.6f} seconds')
print(res, end_time_in_sec-start_time_in_sec)

------------------
Execution Time: 0.000001 seconds
mongo 9.5367431640625e-07
------------------
#include <iostream>
time>ude <c
omanip>e <i

 {d::string say()
    return "mongo";
}

int main() {
in_sec = static_cast<double>(std::clock()) / CLOCKS_PER_SEC;
 = say();string res
sec = static_cast<double>(std::clock()) / CLOCKS_PER_SEC;
Execution Time: " << std::fixed << std::setprecision(6) << (end_time_in_sec - start_time_in_sec) << " seconds" << std::endl;
 " << end_time_in_sec - start_time_in_sec << std::endl;
    return 0;
None
------------------
Execution Time: 0.000004 seconds
mongo 0.000004



In [101]:
cpp='''
#include <iostream>
#include <chrono>

std::string echo() {
    return "apple";
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    auto result = echo();
    auto end_time = std::chrono::high_resolution_clock::now();

    std::cout << "Result: " << result << std::endl;
    auto duration = std::chrono::duration_cast<std::chrono::microseconds>(end_time - start_time).count();
    std::cout << "Execution Time: " << duration << " microseconds" << std::endl;

    return 0;
}
'''
def write_output(x):
    code = x.replace('```cpp','').replace('```','')
    with open('code.cpp', 'w') as f:
        f.write(code)
def run_cpp():
        try:
            run1 = subprocess.run(['clang++', '-Ofast', '-std=c++17', '-march=armv8.5-a', '-mtune=apple-m1', '-mcpu=apple-m1', '-o', 'code', 'code.cpp'], check=True, text=True, capture_output=True)
            run2 = subprocess.run(['./code'], check=True, text=True, capture_output=True)
            return run2.stdout
        except subprocess.CalledProcessError as e:
            return f'An error occurred:\n{e.stderr}'
write_output(cpp)
run_cpp()

'Result: apple\nExecution Time: 0 microseconds\n'

In [19]:
import time
def speak():
    return 'hi'
start_time = time.time()
res = speak()
end_time = time.time()
print('Result:', res)
print(f'Running time: {(end_time-start_time):.6f}s')
# print(f"Execution Time: {(end_time - start_time):.6f} seconds")

Result: hi
Running time: 0.000044s


In [22]:
cppcode='''
#include <stdio.h>

int main() {
    printf("Hello World\n");
    return 0;
}
''' 
def write_output(x):
    code = x.replace("```cpp","").replace("```","")
    with open("generated_cpp.cpp", "w") as f:
        f.write(code)
def execute_cpp(x):
        write_output(x)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "generated_cpp", "generated_cpp.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./generated_cpp"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"
print('Execute C++:',execute_cpp(cppcode))

Execute C++: An error occurred:
generated_cpp.cpp:5:12: warning: missing terminating '"' character [-Winvalid-pp-token]
    printf("Hello World
           ^
generated_cpp.cpp:5:12: error: expected expression
generated_cpp.cpp:6:1: warning: missing terminating '"' character [-Winvalid-pp-token]
");
^
generated_cpp.cpp:8:2: error: expected '}'
}
 ^
generated_cpp.cpp:4:12: note: to match this '{'
int main() {
           ^
2 warnings and 2 errors generated.



In [28]:
import time
code1 = '''
def count(x,y):
     a = []
     for z in range(x,y+1):
          a.append(z)
     return str(a)
start_time = time.time()
result = count(1,10)
end_time = time.time()
print("Result:", result)
print("Execution Time: {:.6f}s".format(end_time - start_time))
'''
code2 = '''
print('hi')
'''
code3 = '''
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
'''
exec(code3)

Result: 3.141592658589
Execution Time: 5.770024 seconds


In [12]:
!clang++ -O3 -std=c++17 -march=armv8.3-a -o generated_cpp generated_cpp.cpp
!./generated_cpp

Hello World

In [23]:
cpp_code1='''
#include <stdio.h>

int main() {
    printf("Hello World");
    return 0;
}

'''
cpp_code2='''
#include <stdio.h>

int main() {
    printf("test test");
    return 0;
}

'''
def write_output(x):
    code = x.replace("```cpp","").replace("```","")
    with open("generated_cpp.cpp", "w") as f:
        f.write(code)
def execute_cpp(x):
        write_output(x)
        try:
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "generated_cpp", "generated_cpp.cpp"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./generated_cpp"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"
execute_cpp(cpp_code2)

'test test'

In [4]:
import subprocess
# 1 write ['ls', 'la'] at test.txt
# with open('test.txt','w') as f:
#     p1 = subprocess.run(['ls','-la'], stdout=f, text=True)
# 2 read texst.txt
p1 = subprocess.run(['more text.txt'], text=True, shell=True, check=True)
print(p1.stdout)


today
is
sunny
day
!!None
